In [ ]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split
import GPUtil
import itertools
import pickle
from datetime import datetime
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import torch
import torch.nn as nn
from sklearn.preprocessing import Normalizer
import joblib
import ili  # Import ili for the SBI functionality
from ili.dataloaders import NumpyLoader
from ili.inference import InferenceRunner
from ili.validation.metrics import PosteriorCoverage, PlotSinglePosterior

from sbi.utils.user_input_checks import process_prior

sys.path.append("/disk/xray15/aem2/camels/proj2")
from setup_params_1P import plot_uvlf, plot_colour
from setup_params_SB import *
# from setup_params import *
from priors_SB import initialise_priors_SB28
from variables_config_28 import uvlf_limits, n_bins_lf, colour_limits, n_bins_colour
# parameters
device = "cuda" if torch.cuda.is_available() else "cpu"
model = "IllustrisTNG"
spec_type = "attenuated"
sps = "BC03"
snap = ["044"]
bands = "all" # or just GALEX?

colours = False  
luminosity_functions = True
name = f"{model}_{bands}_{sps}_{spec_type}_{n_bins_lf}_{n_bins_colour}"

cam = camels(model=model, sim_set='SB28')

if colours and not luminosity_functions:
    model_out_dir = "/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/models/colours_only/"
    plots_out_dir = "/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/sbi_plots/colours_only/"
    
elif luminosity_functions and not colours:
    model_out_dir = "/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/models/lf_only/"
    plots_out_dir = "/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/sbi_plots/lfs_only/"

elif colours and luminosity_functions:
    model_out_dir = "/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/models/colours_lfs/"
    plots_out_dir = "/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/sbi_plots/colours_lfs/"

else:
    raise ValueError("At least one of colours or luminosity_functions must be True")

print("Saving model in ", model_out_dir)
print("Saving plots in ", plots_out_dir)



In [2]:
# First, let's check what type of data we have
def convert_to_numpy(data):
    """
    Safely converts data to numpy array regardless of whether it's a torch tensor or numpy array
    
    Parameters:
        data: Input data (either torch.Tensor or numpy.ndarray)
    
    Returns:
        numpy.ndarray: The data converted to numpy array
    """
    if torch.is_tensor(data):
        # If it's a torch tensor, move to CPU and convert to numpy
        return data.cpu().numpy()
    elif isinstance(data, np.ndarray):
        # If it's already a numpy array, return as is
        return data
    else:
        # If it's neither, try converting to numpy array
        return np.array(data)


In [3]:
# First, let's check what type of data we have
def convert_to_numpy(data):
    """
    Safely converts data to numpy array regardless of whether it's a torch tensor or numpy array
    
    Parameters:
        data: Input data (either torch.Tensor or numpy.ndarray)
    
    Returns:
        numpy.ndarray: The data converted to numpy array
    """
    if torch.is_tensor(data):
        # If it's a torch tensor, move to CPU and convert to numpy
        return data.cpu().numpy()
    elif isinstance(data, np.ndarray):
        # If it's already a numpy array, return as is
        return data
    else:
        # If it's neither, try converting to numpy array
        return np.array(data)


In [ ]:
print(uvlf_limits)

In [ ]:

# parameter info file (df_info) is used for defining priors
# the actual parameter values come from the camels class which reads CosmoAstroSeed_IllustrisTNG_L25n256_SB28.txt

#  parameters defined here: /disk/xray15/aem2/data/28pams/IllustrisTNG/SB/CosmoAstroSeed_IllustrisTNG_L25n256_SB28.txt which is used for theta
df_pars = pd.read_csv('/disk/xray15/aem2/data/28pams/IllustrisTNG/SB/CosmoAstroSeed_IllustrisTNG_L25n256_SB28.txt', delim_whitespace=True)
print(df_pars)


In [ ]:
df_pars

In [ ]:
no_pams = 6
end_of_pams = no_pams + 1 
param_names = df_pars.columns[1:end_of_pams].tolist()  # Excluding 'name' column
param_length = len(param_names)


param_length


In [ ]:
theta_lst = df_pars.iloc[:, 1:end_of_pams].to_numpy()  # excluding 'name' column and 'seed' column

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, param_name in enumerate(param_names):
    axes[i].hist(theta_lst[:, i], bins=50)
    axes[i].set_title(param_name)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Count')

plt.tight_layout()
plt.show()


In [ ]:


# prior values come from this:
df_info = pd.read_csv("/disk/xray15/aem2/data/28pams/Info_IllustrisTNG_L25n256_28params.txt")
print(df_info)


In [ ]:
df_info6 = df_info[0:6]
df_info6

In [ ]:
# overwrite theta with the real theta
if __name__ == "__main__":
    theta, x = get_theta_x_SB(
        luminosity_functions=luminosity_functions,
        colours=colours,
        uvlf_limits=uvlf_limits,
        colour_limits=colour_limits,
        n_bins_lf=n_bins_lf,
        n_bins_colour=n_bins_colour
    )
    print(theta.shape, x.shape)

if colours:
    fig = plot_colour(x)
    plt.savefig('/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/colours_test/colour_check.png')
    plt.show()

if luminosity_functions:
    fig = plot_uvlf(x)
    plt.savefig('/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/LFs_test/uvlf_check.png')
    plt.show()


# get the priors and data
prior = initialise_priors_SB28(
    df=df_info6, 
    device=device,
    astro=True,
    dust=False  # no dust for testing. set to False to only get the 28 model parameters.
    # with dust = True, prior has 32 dimensions (28 parameters + 4 dust parameters) 
)

# process the data
x_all = np.array([np.hstack(_x) for _x in x])
x_all = torch.tensor(x_all, dtype=torch.float32, device=device)

print("Theta shape:", theta.shape)
print("X shape:", x_all.shape)


# Move data to GPU as early as possible
x_all = x_all.to(device)
print('x_all:', x_all)

theta = torch.tensor(theta, dtype=torch.float32, device=device)
print('theta:', theta)

# Handle NaN values and normalize while on GPU
x_all_cpu = x_all.cpu().numpy()  # Only move to CPU when necessary for sklearn
print('x_all_cpu:', x_all_cpu)

print("Data shape before processing:", x_all_cpu.shape)
print("Number of values:",(x_all_cpu).sum())
print("Number of NaN values:", np.isnan(x_all_cpu).sum())
print("Number of infinite values:", np.isinf(x_all_cpu).sum())


# get rid of NaN/inf values, replace with small random noise
nan_mask = np.isnan(x_all_cpu) | np.isinf(x_all_cpu)
print('nan_mask:', nan_mask)


if nan_mask.any():
    x_all_cpu[nan_mask] = np.random.rand(np.sum(nan_mask)) * 1e-10

print("Data shape before processing:", x_all_cpu.shape)
print("Number of NaN values:", np.isnan(x_all_cpu).sum())
print("Number of infinite values:", np.isinf(x_all_cpu).sum())

print('x_all_cpu:', x_all_cpu)

# make test mask
test_mask = create_test_mask() # 10% testing
train_mask = ~test_mask # 90% for training



In [12]:
# # additional processing to solve cpu / cuda issues
# # we want it to be numpy before we put into the model and then put it into the model

# # # process the data
# x_all = np.array([np.hstack(_x) for _x in x])
# # x_all = torch.tensor(x_all, dtype=torch.float32, device=device)

# print("Theta shape:", theta.shape)
# print("X shape:", x_all.shape)


# # # # Move data to GPU as early as possible
# # x_all = x_all.to(device)
# # # print('x_all:', x_all)

# # theta = torch.tensor(theta, dtype=torch.float32, device=device)
# # # print('theta:', theta)

# # Handle NaN values and normalize while on GPU



If using only luminosity functions (LFs):

2 GALEX filters (FUV, NUV)
Each filter gets n_bins_lf (12) bins
2 filters * 12 bins = 24 features

If also using colors:

1 color (FUV-NUV)
Gets n_bins_colour (12) bins
Total would be 24 + 12 = 36 features

In [13]:

# print("Theta shape:", theta.shape)
# print("X shape:", x_all.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

class GlobalScaler:
    def fit_transform(self, x):
        self._mean_x = x.mean()
        self._std_x = x.std()
        return (x - self._mean_x) / self._std_x

    def transform(self, x):
        return (x - self._mean_x) / self._std_x

    def inverse_transform(self, x_norm):
        return (x_norm * self._std_x) + self._mean_x


x_scaler = GlobalScaler()  # scales all features by same mean and std  over N samples
theta_scaler = StandardScaler()  # scales each feature by its own mean and std over N samples

# Convert theta tensors to numpy for scaling
# convert to numpy
x = convert_to_numpy(x)
theta = convert_to_numpy(theta)

print("Type of x:", type(x))
print("Type of theta:", type(theta))

# split into training and testing samples:

# make test mask
test_mask = create_test_mask() # 10% testing
train_mask = ~test_mask # 90% for training


# split data
x_train=x_all_cpu[train_mask]
theta_train=theta[train_mask]

x_test = x_all_cpu[test_mask]
theta_test = theta[test_mask]


# apply scaling /normalisation
x_train = x_scaler.fit_transform(x_train)
x_test = x_scaler.transform(x_train)
# theta_train_scaled = theta_scaler.fit_transform(theta_train_np)
# theta_test_scaled = theta_scaler.transform(theta_test_np)
theta_train = theta_scaler.fit_transform(theta_train)
theta_test = theta_scaler.transform(theta_test)


In [ ]:
if colours:
    fig = plot_colour(x_train)
    plt.savefig('/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/colours_test/colour_check.png')
    plt.show()

if luminosity_functions:
    fig = plot_uvlf(x_train)
    plt.savefig('/disk/xray15/aem2/plots/28pams/IllustrisTNG/SB/test/LFs_test/uvlf_check.png')
    plt.show()


In [35]:
train_args = {
    "training_batch_size": 16,     # Slightly reduced
    "learning_rate": 1e-4,        # Original learning rate for fewer parameters
    "stop_after_epochs": 20,      # More patience
    "clip_max_norm": 2,        # Tight gradient control
     "validation_fraction": 0.1,    # More validation data
    "max_num_epochs": 200,       # Allow longer training
    "show_train_summary": True
}

# Slightly increased capacity for better learning of core parameters
hidden_features = 30
num_transforms = 2
num_nets = 1

# refer back to training args in file name
model_params = f"batch{train_args['training_batch_size']}_" \
               f"lr{train_args['learning_rate']:.0e}_" \
               f"epochs{train_args['stop_after_epochs']}_" \
               f"val{train_args['validation_fraction']}_" \
               f"hidden{hidden_features}_" \
               f"transforms{num_transforms}"

# Create neural networks
nets = [ili.utils.load_nde_sbi(
    engine="NPE",
    model="nsf",
    hidden_features=hidden_features,
    num_transforms=num_transforms,
) for _ in range(num_nets)]

# Modify loader to use core parameters
loader = NumpyLoader(
    x=x_train,#.clone().detach(),
    theta=theta_train#.clone().detach()
    
)


# Update model output directory for phase 1
# model_out_dir_phase1 = os.path.join(model_out_dir, "phase1_core")
# os.makedirs(model_out_dir_phase1, exist_ok=True)

# Create inference runner with core parameters
runner = InferenceRunner.load(
    backend="sbi",
    engine="NPE",
    prior=prior,
    nets=nets,
    device=device,
    train_args=train_args,
    proposal=None,
    out_dir=model_out_dir,
    # name=name + "_phase1_core"
)



In [ ]:
# After training, update plotting to show only core parameters
posterior_ensemble, summaries = runner(loader=loader)


In [ ]:
posterior_ensemble

In [ ]:
summaries

In [ ]:
def plot_training_diagnostics(summaries):
    """Plot training diagnostics without empty subplots"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))  # Changed to 1 row, 2 columns
    
    # Loss curves
    train_losses = summaries[0]['training_log_probs']
    val_losses = summaries[0]['validation_log_probs']
    epochs = range(len(train_losses))
    
    ax1.plot(epochs, train_losses, '-', label='Training', color='blue')
    ax1.plot(epochs, val_losses, '--', label='Validation', color='red')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Log probability')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Overfitting gap
    gap = np.array(train_losses) - np.array(val_losses)
    ax2.plot(epochs, gap, '-', color='purple')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss difference')
    ax2.set_title('Overfitting Gap')
    ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Use the function
fig = plot_training_diagnostics(summaries)
plt.savefig(os.path.join(plots_out_dir, f'loss_overfitting_{model_params}.png'))
plt.show()
plt.close()

In [ ]:
# plot train/validation loss
fig, ax = plt.subplots(1, 1, figsize=(6,4))
c = list(mcolors.TABLEAU_COLORS)
for i, m in enumerate(summaries):
    ax.plot(m['training_log_probs'], ls='-', label=f"{i}_train", c=c[i])
    ax.plot(m['validation_log_probs'], ls='--', label=f"{i}_val", c=c[i])
ax.set_xlim(0)
ax.set_xlabel('Epoch')
ax.set_ylabel('Log probability')
ax.legend()

### Drawing samples from the entire test set to look at overall performance of the model

In [ ]:
type(theta_test)

In [ ]:
type(x_test)

In [31]:
# After scaling, convert to torch tensors with correct dtype
# x_train = torch.tensor(x_train, dtype=torch.float32, device=device) 
x_test = torch.tensor(x_test, dtype=torch.float32, device=device)
# theta_train = torch.tensor(theta_train, dtype=torch.float32, device=device)
theta_test = torch.tensor(theta_test, dtype=torch.float32, device=device)


In [ ]:
type(theta_test)

In [ ]:
type(x_test)

In [ ]:

# Get test data
# x_test = x_all[test_mask]
# theta_test = theta[test_mask]

# we already have test data and it has been scaled already:
# try this instead
# x_test = convert_to_numpy(x_test).cpu()
# theta_test = convert_to_numpy(theta_test).cpu()

# Number of samples to draw from posterior
n_samples = 1000

# Storage for predictions
all_samples = []
all_means = []
all_stds = []

# Suppress the deprecation warning
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='nflows.transforms.lu')

# Generate posterior samples for each test point
for i in range(len(x_test)):
    # Get samples from the posterior
    with torch.no_grad():  # Add this for efficiency
        samples = posterior_ensemble.sample(
            (n_samples,), 
            x=x_test[i].reshape(1, -1)
        ).cpu().numpy()
    
    # Calculate mean and std of samples
    mean = samples.mean(axis=0)
    std = samples.std(axis=0)
    
    all_samples.append(samples)
    all_means.append(mean)
    all_stds.append(std)

all_samples = np.array(all_samples)
all_means = np.array(all_means)
all_stds = np.array(all_stds)

In [ ]:

# # Number of samples to draw from posterior
# n_samples = 1000

# # Storage for predictions
# all_samples = []
# all_means = []
# all_stds = []

# # Suppress the deprecation warning
# import warnings
# warnings.filterwarnings('ignore', category=UserWarning, module='nflows.transforms.lu')

# # Generate posterior samples for each test point
# for i in range(len(x_test)):
#     # Get samples from the posterior
#     with torch.no_grad():  # Add this for efficiency
#         samples = posterior_ensemble.sample(
#             (n_samples,), 
#             x=x_test[i].reshape(1, -1)
#         ).cpu().numpy()
    
#     # Calculate mean and std of samples
#     mean = samples.mean(axis=0)
#     std = samples.std(axis=0)
    
#     all_samples.append(samples)
#     all_means.append(mean)
#     all_stds.append(std)

# all_samples = np.array(all_samples)
# all_means = np.array(all_means)
# all_stds = np.array(all_stds)

In [ ]:
param_names = df_pars.columns[1:29].tolist()  # Excluding 'name' column

fig, axes = plt.subplots(7, 4, figsize=(16, 28)) 
axes = axes.flatten()

fontsize = 10  

plt.rcParams['figure.constrained_layout.use'] = True  

# Plot each parameter
for i in range(28):
    ax = axes[i]
    
    # True vs predicted with error bars
    ax.errorbar(
        theta_test[:, i].cpu().numpy(),
        all_means[:, i],
        yerr=all_stds[:, i],
        fmt='.',
        color='k',
        ecolor='blue',
        capsize=0,
        elinewidth=0.8,  
        alpha=0.3,       
        markersize=5    
    )
    
    # Add true line
    lims = [
        min(ax.get_xlim()[0], ax.get_ylim()[0]),
        max(ax.get_xlim()[1], ax.get_ylim()[1])
    ]
    ax.plot(lims, lims, '--', color='black', alpha=0.5, linewidth=1)
    
    # get metrics
    rmse = np.sqrt(np.mean((theta_test[:, i].cpu().numpy() - all_means[:, i])**2))
    r2 = np.corrcoef(theta_test[:, i].cpu().numpy(), all_means[:, i])[0, 1]**2
    chi2 = np.mean(((theta_test[:, i].cpu().numpy() - all_means[:, i])**2) / (all_stds[:, i]**2))
    
    # add metrics box in top left corner
    stats_text = f'RMSE = {rmse:.2f}\n' + \
                 f'R² = {r2:.2f}\n' + \
                 f'χ² = {chi2:.2f}'
    ax.text(0.05, 0.95, stats_text,
            transform=ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.8),
            verticalalignment='top',
            fontsize=fontsize-1)  # Slightly smaller font for stats
    
    # title: parameter name
    ax.set_title(param_names[i], fontsize=fontsize, pad=5)  # Reduced padding
    
    # axis labels
    ax.set_xlabel('True', fontsize=fontsize-1)
    ax.set_ylabel('Inferred', fontsize=fontsize-1)
    
    # tick labels
    ax.tick_params(axis='both', which='major', labelsize=fontsize-2)
    
    # internal padding
    ax.margins(x=0.05, y=0.05)

# subplot spacing
plt.subplots_adjust(
    left=0.01,    # Less space on left
    right=0.7,   # Less space on right
    bottom=0.05,  # Less space at bottom
    top=0.7,     # Less space at top
    wspace=0.2,   # Less space between plots horizontally
    hspace=0.2    # Less space between plots vertically
)


# Save figure with detailed filename
save_path = f'{plots_out_dir}/posterior_predictions_{model_params}.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
print(save_path)
plt.show()

In [ ]:
config_str = (f"batch{train_args['training_batch_size']}_"
             f"lr{train_args['learning_rate']}_"
             f"epochs{train_args['stop_after_epochs']}_"
             f"h{hidden_features}_t{num_transforms}")

# coverage plots
metric = PosteriorCoverage(
    num_samples=int(4e3),
    sample_method='direct',
    labels=cam.labels,
    plot_list=["tarp"], # "coverage", "histogram", "predictions", 
    out_dir=plots_out_dir,
)

# Generate plots
figs = metric(
    posterior=posterior_ensemble,
    x=x_all[test_mask].cpu(),
    theta=theta[test_mask, :].cpu(),
    signature=f"coverage_{name}_{config_str}_"  # Add config to filename
)

config_text = (
    f"Training Config:\n"
    f"Batch Size: {train_args['training_batch_size']}\n"
    f"Learning Rate: {train_args['learning_rate']}\n"
    f"Epochs: {train_args['stop_after_epochs']}\n"
    f"Hidden Features: {hidden_features}\n"
    f"Num Transforms: {num_transforms}"
)

# Process each figure
for i, fig in enumerate(figs):
    plt.figure(fig.number)  # Activate the figure
    plt.figtext(0.02, 0.98, config_text,
                fontsize=8,
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='gray'),
                verticalalignment='top')
    
    # Save each figure with type indicator
    plot_types = ["tarp"] #"coverage", "histogram", "predictions",
    plt.savefig(os.path.join(plots_out_dir, 
                f'metric_{plot_types[i]}_{name}_{config_str}.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
def get_performance_metric(summaries, posterior):
    # Get validation loss at convergence
    final_val_loss = summaries[0]['validation_log_probs'][-1]
    
    # Get average RMSE across parameters
    rmse_vals = [
        np.sqrt(np.mean((theta_test[:, i].cpu().numpy() - all_means[:, i])**2))
        for i in range(len(param_names))
    ]
    avg_rmse = np.mean(rmse_vals)
    
    # Get average R² across parameters  
    r2_vals = [
        np.corrcoef(theta_test[:, i].cpu().numpy(), all_means[:, i])[0, 1]**2 
        for i in range(len(param_names))
    ]
    avg_r2 = np.mean(r2_vals)

    # Composite score (lower is better)
    # Weighting validation loss more heavily since it's key for convergence
    composite_score = -final_val_loss + avg_rmse - avg_r2

    return {
        'composite_score': composite_score,
        'val_loss': final_val_loss,
        'avg_rmse': avg_rmse,
        'avg_r2': avg_r2
    }

results = {
    'architecture': {
        'hidden_features': hidden_features,
        'num_transforms': num_transforms,
        'num_nets': num_nets
    },
    'training': {
        'batch_size': train_args['training_batch_size'],
        'learning_rate': train_args['learning_rate'],
        'epochs': train_args['stop_after_epochs']
    },
    'performance': get_performance_metric(summaries, posterior_ensemble)
}

# Flatten the nested dictionary into a single level
flat_results = {
   'hidden_features': results['architecture']['hidden_features'],
   'num_transforms': results['architecture']['num_transforms'], 
   'num_nets': results['architecture']['num_nets'],
   'batch_size': results['training']['batch_size'],
   'learning_rate': results['training']['learning_rate'],
   'epochs': results['training']['epochs'],
   'composite_score': results['performance']['composite_score'],
   'val_loss': results['performance']['val_loss'],
   'avg_rmse': results['performance']['avg_rmse'], 
   'avg_r2': results['performance']['avg_r2']
}

# Create DataFrame with single row
df = pd.DataFrame([flat_results])

# Improve formatting
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(df.to_string(index=False))

In [86]:
import os

def save_results(results):
   flat_results = {
       'hidden_features': results['architecture']['hidden_features'],
       'num_transforms': results['architecture']['num_transforms'], 
       'num_nets': results['architecture']['num_nets'],
       'batch_size': results['training']['batch_size'],
       'learning_rate': results['training']['learning_rate'],
       'epochs': results['training']['epochs'],
       'composite_score': results['performance']['composite_score'],
       'val_loss': results['performance']['val_loss'],
       'avg_rmse': results['performance']['avg_rmse'],
       'avg_r2': results['performance']['avg_r2']
   }
   
   df_row = pd.DataFrame([flat_results])
   
   if os.path.exists('sbi_results_norm.csv'):
       df_row.to_csv('sbi_results_norm.csv', mode='a', header=False, index=False)
   else:
       df_row.to_csv('sbi_results_norm.csv', index=False)
       
save_results(results)

The composite score weights validation loss more heavily since it's crucial for the model's predictive performance. Lower composite scores indicate better overall performance.